### Basic Imports

In [11]:
import numpy as np
import pandas as pd
import os
import sys

from sklearn.feature_extraction.text import CountVectorizer

from sklearn.base import BaseEstimator
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import make_pipeline

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

import matplotlib.pyplot as plt
%matplotlib inline

### Loading Data

In [12]:
#set relative path to data
DATA_PATH = os.path.join("../data/")

#true to load in training data
def load_movie_reviews(path=DATA_PATH, train=True):
    if train:
        tsv_path = os.path.join(path, "labeledTrainData.tsv")
    else:
        tsv_path = os.path.join(path, "testData.tsv")
    return pd.read_csv(tsv_path, delimiter='\t', header=0, quoting=3)

In [13]:
#load train data
review_train_orig = load_movie_reviews(path=DATA_PATH, train=True)
review_train = review_train_orig.copy()

#load test data
review_test_orig = load_movie_reviews(path=DATA_PATH, train=False)
review_test = review_test_orig.copy()

In [14]:
review_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 2 columns):
id        25000 non-null object
review    25000 non-null object
dtypes: object(2)
memory usage: 390.7+ KB


In [15]:
review_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.0+ KB


***Note:*** we need to remember to add in the 'sentiment' for the test data set based on the 'id' column

### Adding 'sentiment' To Test Data

In [16]:
def add_sentiment(df):
    sentiment = [1 if int(x[1].strip('"')) >= 5 else 0 for x in df['id'].str.split('_')]
    df['sentiment'] = sentiment
    return

#adding sentiment column to test data
add_sentiment(review_test)
review_test.head()

,id,review,sentiment
0,"""12311_10""","""Naturally in a film who's main themes are of ...",1
1,"""8348_2""","""This movie is a disaster within a disaster fi...",0
2,"""5828_4""","""All in all, this is a movie for kids. We saw ...",0
3,"""7186_2""","""Afraid of the Dark left me with the impressio...",0
4,"""12128_7""","""A very accurate depiction of small time mob l...",1


### Build Pipeline